In [ ]:
!pip install -qU crewai duckduckgo-search langchain-groq langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip show crewai

Name: crewai
Version: 0.28.8
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, click, embedchain, instructor, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            # api_key=os.getenv("GROQ_API_KEY"),
            model="llama3-70b-8192"
        )

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
search_tool = DuckDuckGoSearchRun()
# search_tool = DuckDuckGoSearchResults()

In [ ]:
search_tool.run("Tell me about GenAI?")

'Generative AI uses its representations extracted from training data to generate new kinds of content in the chosen medium. An image-based generative AI, for example, learns and mirrors common patterns that define human faces. A generative AI model generally works in one or more of the following media: Text. Images. Audio. This is a great way to get your feet wet with GenAI if you already have the right data architecture and data engineering capabilities in place. Visit this page to learn more about our GenAI AppLink service. Call us at 1-800-591-0442. Send us an email at sales@clearscale.com. Fill out a Contact Form. Generative AI, also referred to as GenAI, allows users to input a variety of prompts to generate new content, such as text, images, videos, sounds, code, 3D designs, and other media. It "learns" and is trained on documents and artifacts that already exist online. Generative AI evolves as it continues to train on more data. Frequently Asked Questions About Generative AI. G

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

In [ ]:
# Define agent along with your goals
researcher = Agent(
    role="Research & Developement Engineer",
    goal="Uncover cutting-edge developments in AI and data science",
    backstory="""As an R&D engineer in GenAI, you pioneer AI-driven solutions for genetic analysis.
                 Crafting algorithms and software, you decode genomic data to predict traits and diseases.
                 Collaborating across disciplines, you accelerate research in personalized medicine and agriculture, shaping the future of genetic understanding and application.""",
    llm=GROQ_LLM,
    verbose=True,
    allow_delegation=False
)

system_design_engineer = Agent(
    role='Senior System Design Engineer',
    goal="Give system requirement details (like GPU VRAM, RAM, ROM, and other computation required) for the Machine Learning, Deep Learning, large language models (LLM), and Large Vision Models (LVM)",
    backstory="""You are a renowned Senior System Design Engineer, known for your insightful and engaging outputs.
                 You transform complex concepts into compelling narratives & deliver to user in formatted way.""",
    llm=GROQ_LLM,
    verbose=True,
    allow_delegation=True
)

In [ ]:
# Create task for your agents
task1 = Task(
    description="""Perform an exhaustive examination of the forefront advancements in AI throughout 2024.
    Identify pivotal trends, groundbreaking technologies, and their potential ramifications across industries.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher
)

task2 = Task(
    description="""Using the insights provided, develop an system design document
    that highlights the accurate system requirement details for executing the AI models.
    Your generated document should be informative yet accessible, catering to a tech-savvy audience.
    Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Give full system design details for the running the model in specific way.",
    agent=system_design_engineer
)

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, system_design_engineer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

In [ ]:
# Get your crew to work!
result = crew.kickoff()

print("###########")
print(result)

 [DEBUG]: == Working Agent: Research & Developement Engineer
 [INFO]: == Starting Task: Perform an exhaustive examination of the forefront advancements in AI throughout 2024. 
    Identify pivotal trends, groundbreaking technologies, and their potential ramifications across industries.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: 

**Exhaustive Examination of Forefront Advancements in AI throughout 2024**

**Pivotal Trends:**

• **Explainability and Transparency in AI**: Developing techniques to interpret and explain AI-driven decision-making processes, enhancing trust and accountability in AI systems.

• **Edge AI**: The proliferation of AI capabilities at the edge of the network, reducing latency and improving real-time processing for applications like autonomous vehicles and smart homes.

• **Adversarial Robustness**: Focusing on developing AI systems that can withstand adversarial attacks, ensuring the security and reliability of